In [1]:
import zipfile, os, re
import pandas as pd
import numpy as np 

# Read in the fooball data

In [2]:
_file = os.path.join('ncaa_football_scrapy','Data', 'Data.zip')
zf = zipfile.ZipFile(_file)
filenames = zf.namelist()

# Create Program History DF

In [3]:
history = pd.read_csv(zf.open('history.csv'))

In [4]:
history['year'] = history.apply(lambda x: int(x['Year'][:4]), axis = 1)

# Coach links

In [5]:
coach_links = pd.read_csv(os.path.join("ncaa_football_scrapy", "Links", "links_teaminfo.csv"))
coach_links = coach_links.rename(columns={"txt":"coach", 'team':"Team"})
coach_links = coach_links[coach_links.key == 'people'][['Team', 'coach', 'year']]

# Create Coach DF 

In [6]:
coaches = pd.read_csv(zf.open('coaches.csv'))

In [7]:
coaches['year'] = coaches.apply(lambda x: int(x['Year'][:4]), axis = 1)
coaches.rename(columns = {'Name':'coach'}, inplace = True)

# Create Roster DF

In [8]:
roster = pd.read_csv(zf.open('roster.csv'))

In [9]:
roster['year'] = roster.apply(lambda x: int(x['Year'][:4]), axis = 1)

In [10]:
def yrRecode(data): 
    yr = {'Fr':0, 'So':1, 'Jr':2, 'Sr':3}
    return yr[data]

In [11]:
roster_by_year = roster[['Team', 'Yr', 'year', 'GS']].groupby(['Team', 'year', 'Yr']).count().reset_index()

In [12]:
roster_games_started = roster[['Team', 'year', 'Yr', 'GS']].groupby(['Team', 'year', "Yr"]).mean().reset_index()

In [13]:
roster_games_played = roster[['Team', 'year', 'Yr', 'GP']].groupby(['Team', 'year', "Yr"]).mean().reset_index()

In [14]:
roster_final = pd.merge(roster_games_played, 
                        roster_games_started, 
                        left_on = ['Team', 'year', 'Yr'], 
                        right_on = ['Team', 'year', 'Yr'],
                        how = 'left'
                       )

In [15]:
roster_final2 = roster_final.set_index(['Team', 'year', 'Yr']).unstack().reset_index()

In [16]:
columns = [x[0] if x[0] in ['Team', 'year'] else x[0] + "_" + x[1] for x in roster_final2.columns.values]
roster_final2.columns = columns

In [17]:
roster_final2.head(3)

,Team,year,GP_Fr,GP_Jr,GP_So,GP_Sr,GS_Fr,GS_Jr,GS_So,GS_Sr
0,Air Force Falcons,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
1,Air Force Falcons,2014,1.000000,8.000000,6.363636,9.900000,0.000000,3.840000,1.575758,4.466667
2,Air Force Falcons,2015,1.800000,9.555556,6.714286,9.142857,0.000000,4.611111,0.885714,5.047619


In [18]:
def createvariables(data):
    """Create Opponent, Home, Win/Loss, Overtime, and Scores
    
    """
    regexs = {'WL': "[WL]",
              'team': ["\d+\s\-", "\d+"],
              'opponent_score':["\-\s\d+", '\d+'],
              'OT':["\(\d+OT\)", "\d+"], 
              'opponent':['[\w\s]+\.?\s\@\s\w+']
             }
    
    # Opponent 
    if '@' in data['Opponent']: 
        search = re.search(regexs['opponent'][0], data['Opponent']) 
        if search: 
            opponent = data['Opponent'].split("@")[0].strip()
            home = 0
        else: 
            opponent = data['Opponent'].replace("@","").strip()
            home = 0
    else: 
        opponent = data['Opponent']
        home = 1
    
    
    # Win/Loss
    WinLoss = re.search(regexs['WL'], data['Result']).group()
    
    # Overtime 
    Overtime = 0
    re_ot = re.search(regexs['OT'][0], data['Result'])
    if re_ot:
        Overtime = re.search(regexs['OT'][1], re_ot.group()).group()
        
    # Team Score
    team_score = None
    re_team = re.search(regexs['team'][0], data['Result'])
    if re_team:
        team_score = re.search(regexs['OT'][1], re_team.group()).group()
        
    # Opponent Score
    opponent_score = None
    re_opponent = re.search(regexs['opponent_score'][0], data['Result'])
    if re_opponent:
        opponent_score = re.search(regexs['OT'][1], re_opponent.group()).group()
        
    return pd.Series([opponent, home, WinLoss, Overtime, team_score, opponent_score])    

# Start merging

In [19]:
# Subset the gamebygame files
targetfiles = [file for file in filenames if 'gamebygame' in file ]

In [20]:
# Create a dict of dfs and create a multi-index 
dfs = {}
for f in targetfiles: 
    filename = f.split(".")[0]
    dfs[filename] = pd.read_csv(zf.open(f))
    dfs[filename]['Date'] = pd.to_datetime(dfs[filename]['Date'])
    dfs[filename].set_index(['Team', 'Date', 'OffenseDefense'], inplace = True)

In [21]:
# Concatenate the data together and drop duplicate column names 
master = pd.concat([v for k, v in dfs.items()], axis = 1)
master = master.loc[:, ~master.columns.duplicated()]

In [22]:
# Create new variables 
master[['opponent', 'home', 'WinLoss','Overtime', 'team_score', 'opponent_score']] = master.apply(lambda x: createvariables(x), axis = 1)

In [23]:
master[master.index.get_level_values('OffenseDefense') == 'Offense']

Opponent  \
Team              Date       OffenseDefense                                 
Air Force Falcons 2013-08-31 Offense                              Colgate   
                  2013-09-07 Offense                             Utah St.   
                  2013-09-13 Offense                          @ Boise St.   
                  2013-09-21 Offense                              Wyoming   
                  2013-09-28 Offense                             @ Nevada   
                  2013-10-05 Offense                               @ Navy   
                  2013-10-10 Offense                        San Diego St.   
                  2013-10-26 Offense                           Notre Dame   
                  2013-11-02 Offense                      Army West Point   
                  2013-11-08 Offense                         @ New Mexico   
                  2013-11-21 Offense                                 UNLV   
                  2013-11-30 Offense                       @ Colorado St.   
                  2014-08-30 Offense                         Nicholls St.   
                  2014-09-06 Offense                            @ Wyoming   
                  2014-09-13 Offense                        @ Georgia St.   
                  2014-09-27 Offense                            Boise St.   
                  2014-10-04 Offense                                 Navy   
                  2014-10-11 Offense                           @ Utah St.   
                  2014-10-18 Offense                           New Mexico   
                  2014-11-01 Offense                    @ Army West Point   
                  2014-11-08 Offense                               @ UNLV   
                  2014-11-15 Offense                               Nevada   
                  2014-11-21 Offense                      @ San Diego St.   
                  2014-11-28 Offense                         Colorado St.   
                  2014-12-20 Offense         Western Mich. @ Boise, Idaho   
                  2015-09-05 Offense                           Morgan St.   
                  2015-09-12 Offense                         San Jose St.   
                  2015-09-19 Offense                       @ Michigan St.   
                  2015-10-03 Offense                               @ Navy   
                  2015-10-10 Offense                              Wyoming   
...                                                                   ...   
Wyoming Cowboys   2014-11-01 Offense                         @ Fresno St.   
                  2014-11-07 Offense                             Utah St.   
                  2014-11-22 Offense                            Boise St.   
                  2014-11-29 Offense                         @ New Mexico   
                  2015-09-05 Offense                         North Dakota   
                  2015-09-12 Offense                        Eastern Mich.   
                  2015-09-19 Offense                     @ Washington St.   
                  2015-09-26 Offense                           New Mexico   
                  2015-10-03 Offense                    @ Appalachian St.   
                  2015-10-10 Offense                          @ Air Force   
                  2015-10-17 Offense                               Nevada   
                  2015-10-24 Offense                          @ Boise St.   
                  2015-10-30 Offense                           @ Utah St.   
                  2015-11-07 Offense                         Colorado St.   
                  2015-11-14 Offense                      @ San Diego St.   
                  2015-11-28 Offense                                 UNLV   
                  2016-09-03 Offense                        Northern Ill.   
                  2016-09-10 Offense                           @ Nebraska   
                  2016-09-17 Offense                             UC Davis   
                  2016-09-23 Offense                      @ E

In [24]:
master.reset_index(inplace = True)

In [25]:
master['year'] = master['Date'].dt.year

In [26]:
roster_final2.head()

,Team,year,GP_Fr,GP_Jr,GP_So,GP_Sr,GS_Fr,GS_Jr,GS_So,GS_Sr
0,Air Force Falcons,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
1,Air Force Falcons,2014,1.000000,8.000000,6.363636,9.900000,0.000000,3.840000,1.575758,4.466667
2,Air Force Falcons,2015,1.800000,9.555556,6.714286,9.142857,0.000000,4.611111,0.885714,5.047619
3,Air Force Falcons,2016,9.000000,6.545455,3.848485,10.484848,0.000000,2.454545,0.212121,5.939394
4,Akron Zips,2013,2.000000,7.592593,9.043478,10.090909,0.241379,2.851852,3.173913,4.772727


In [27]:
tmp = pd.merge(master,
               roster_final2, 
               left_on = ['Team', 'year'], 
               right_on = ['Team', 'year'],
               how = 'left'
              )

In [28]:
def teamhistory(team, year, duration): 
    global history
    team = team
    target_year = year
    out = []
    # Individual 
    for yr in duration: 
        target_year_min = target_year - yr
        # Wins/Losses
        wins, losses = list(history[(history.Team == team) & 
                                    (history.year <= target_year) & 
                                    (history.year >= target_year_min)][['Wins', 'Losses']].sum()
                           )
        out.extend([wins, losses, wins/(wins+losses)])
    # Max 
    wins, losses = list(history[(history.Team == team)][['Wins', 'Losses']].sum())
    out.extend([wins, losses, wins/(wins+losses)])
    return pd.Series(out)

In [29]:
# Add in team historical data 
yrs = [1, 2, 3, 5, 10]
variables = ['history_wins_{yr}yrs', 'history_losses_{yr}yrs', 'history_WL_{yr}yrs']
years = [v.format(yr = yr) for yr in yrs for v in variables ]
years.extend([v.format(yr = 'max') for v in variables])

tmp[years] = tmp.apply(lambda x: teamhistory(x['Team'], x['year'], yrs), axis = 1)

In [30]:
# Add in coach info 
tmp = tmp.merge(coach_links, 
                left_on = ['Team', 'year'], 
                right_on = ['Team', 'year'], 
               )

In [33]:
def coach_history(coach, year): 
    global coaches
    coach_record = coaches[(coaches.coach == coach) & 
                           (coaches.year < year)
                          ][["coach","Wins","Losses","WL","year"]].groupby("coach").agg(['sum', 'count', 'mean'])
    coach_record.columns = [x[0] + "_" + x[1]  for x in coach_record.columns.values]
    coach_record.drop(['Wins_count', 'Wins_mean', 'Losses_count', 'Losses_mean', 'WL_sum', 'WL_count', 
                             'year_sum', 'year_mean'], axis=1, inplace = True)
    coach_record.rename(columns = {'Wins_sum':"Coach_wins", 
                                   "Losses_sum":"Coach_losses", 
                                   "WL_mean":"Coach_WL", 
                                   "year_count":"Coach_years"
                                  },
                        inplace = True
                       )
    return pd.Series(coach_record.to_dict(orient='record')[0])

In [34]:
coach_variables = ['Coach_wins', 'Coach_losses', 'Coach_WL', 'Coach_years']
tmp[coach_variables] = tmp.apply(lambda x: coach_history(x['coach'], x['year']), axis = 1)

In [35]:
tmp.Team.unique()

array(['Air Force Falcons', 'Appalachian St. Mountaineers',
       'Arizona St. Sun Devils', 'Arkansas St. Red Wolves',
       'Army West Point Black Knights', 'Auburn Tigers', 'BYU Cougars',
       'Ball St. Cardinals', 'Baylor Bears', 'Boise St. Broncos',
       'Boston College Eagles', 'Charlotte 49ers', 'Clemson Tigers',
       'Colorado St. Rams', 'Duke Blue Devils', 'East Carolina Pirates',
       'Eastern Mich. Eagles', 'FIU Panthers', 'Fla. Atlantic Owls',
       'Florida St. Seminoles', 'Fresno St. Bulldogs',
       'Ga. Southern Eagles', 'Georgia St. Panthers',
       'Georgia Tech Yellow Jackets', 'Iowa St. Cyclones',
       'Kansas St. Wildcats', 'Kent St. Golden Flashes',
       'LSU Fighting Tigers', 'La.-Monroe Warhawks',
       "Louisiana Ragin' Cajuns", 'Louisiana Tech Bulldogs',
       'Miami (FL) Hurricanes', 'Miami (OH) RedHawks',
       'Michigan St. Spartans', 'Middle Tenn. Blue Raiders',
       'Mississippi St. Bulldogs', 'NC State Wolfpack', 'Navy Midshipmen',
 